# Guardrails Contextual Grounding과 Knowledge Base 결합하기
이 노트북은 Amazon Bedrock Guardrails의 contextual grounding 필터를 Amazon Bedrock Knowledge Bases와 결합하는 방법을 보여 줍니다. 이렇게 하면 모델 응답이 Knowledge Base에 저장된 정보에 기반해 사실적으로 정렬되도록 보장할 수 있습니다.

이 노트북에서는 [D&D Systems Reference Document(SRD)](https://www.dndbeyond.com/resources/1781-systems-reference-document-srd)(CC-BY-4.0)을 Knowledge Base에 저장해 사용합니다.  

이 노트북에서 수행할 작업:
1. **라이브러리 임포트:** 노트북이 올바르게 동작하도록 필요한 라이브러리를 로드합니다. 
2. **Knowledge Base 생성:** D&D Systems Reference Document를 Amazon Bedrock의 관리형 Knowledge Base에 저장합니다. 
3. **Guardrail 구성:** contextual grounding 필터 임계값으로 Guardrail을 구성합니다.
4. **Contextual grounding 테스트:** Knowledge Base에서 컨텍스트를 검색해 질의와 함께 LLM에 전달하고 환각 여부를 평가합니다. 
5. **리소스 삭제:** 비용 절감을 위해 생성한 모든 리소스를 삭제합니다. 

<div class="alert alert-block alert-info">
<b>참고:</b> 이 노트북은 `Anthropic Claude 3 Sonnet`과 `Titan Embedding Text V2` 모델을 사용하므로 Amazon Bedrock 콘솔에서 해당 모델에 대한 접근 권한을 활성화하세요.
</div>

## 1. 라이브러리 임포트

In [ ]:
%pip install --upgrade pip --quiet
%pip install -r ../requirements.txt --no-deps --quiet
%pip install -r ../requirements.txt --upgrade --quiet

이 코드는 설정 과정의 일부이며 다음 작업을 수행합니다:
- 상위 디렉터리를 Python 시스템 경로에 추가합니다.
- 이후 실행에 필요한 커스텀 모듈인 `utils`의 `BedrockStructuredKnowledgeBase`를 가져옵니다.

In [ ]:
import os
import sys
import time
import boto3
import logging
import pprint
import json
from pathlib import Path
import requests
import uuid

current_path = Path().resolve()
current_path = current_path.parent

if str(current_path) not in sys.path:
    sys.path.append(str(current_path))

# Print sys.path to verify
print(sys.path)

from utils.knowledge_base import BedrockKnowledgeBase
from utils.knowledge_base_operators import print_results,print_results_with_guardrail

In [3]:
session = boto3.session.Session()
region = session.region_name
unique_id = str(uuid.uuid4())[:4]
s3_client = boto3.client("s3",region_name=region)
bedrock = boto3.client("bedrock",region_name=region)
bedrock_runtime = boto3.client("bedrock-runtime",region_name=region)
bedrock_agent_client = boto3.client("bedrock-agent",region_name=region)
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime",region_name=region)

## 2. Knowledge Base 생성

### 2.1 데이터 세트 다운로드

In [ ]:
url = "https://media.wizards.com/2023/downloads/dnd/SRD_CC_v5.1.pdf"
file_name = "kb_documents/SRD_CC_v5.1.pdf"
os.makedirs("kb_documents", exist_ok=True)
response = requests.get(url)
with open(file_name, "wb") as file:
    file.write(response.content)
print(f"File '{file_name}' has been downloaded.")

### 2.1 Knowledge Base 생성

이제 Knowledge Base와 필요한 리소스를 생성합니다. 여기에는 다음이 포함됩니다:
- 벡터 데이터베이스용 [Amazon OpenSearch Serverless](https://aws.amazon.com/opensearch-service/features/serverless/)
- [AWS IAM](https://aws.amazon.com/iam/) 역할과 권한
- Knowledge Base 문서를 저장할 [Amazon S3](https://aws.amazon.com/s3/) 버킷

Knowledge Base와 종속 리소스를 생성하기 위해 이 폴더에 있는 `BedrockKnowledgeBase` 지원 클래스를 사용합니다. 이 클래스는 새 Knowledge Base를 만들고, 데이터 소스에 문서를 적재하고, 실습을 마친 후 리소스를 삭제할 수 있는 기능을 제공합니다.

In [5]:
knowledge_base_name = "{}-cgdemo".format(unique_id)
knowledge_base_description = "Knowledge Base containing d&d Guide"
bucket_name = "{}-cgdemo-bucket".format(unique_id)

#### 데이터 소스 정의

Knowledge Base에 연결할 데이터 소스를 정의합니다. Amazon Bedrock Knowledge Bases에서는 S3, SharePoint, Salesforce, Confluence 등 다양한 데이터 소스를 선택할 수 있습니다. 이 노트북에서는 S3 버킷을 데이터 소스로 사용합니다.

In [6]:
data_sources=[{"type": "S3", "bucket_name": bucket_name}]

In [ ]:
knowledge_base = BedrockKnowledgeBase(
    kb_name=knowledge_base_name,
    kb_description=knowledge_base_description,
    data_sources=data_sources,
    chunking_strategy = "FIXED_SIZE", 
    suffix = f'{unique_id}-f'
)

이제 Knowledge Base 문서를 S3에 업로드합니다.

In [ ]:
def upload_directory(path, bucket_name):
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".pdf"):
                file_to_upload = os.path.join(root, file)
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload, bucket_name, file)

upload_directory("kb_documents", bucket_name)

그리고 문서를 Knowledge Base에 적재합니다.

In [ ]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base.start_ingestion_job()

In [ ]:
kb_id = knowledge_base.get_knowledge_base_id()

### 2.2 Knowledge Base 테스트
생성한 Knowledge Base가 예상대로 동작하는지 확인해 보겠습니다. 먼저 Knowledge Base ID를 가져옵니다. 

그런 다음 [`RetrieveAndGenerate`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) API를 사용해 Knowledge Base에서 질문에 대한 컨텍스트를 검색하고 최종 응답을 생성합니다.

In [ ]:
model_id="anthropic.claude-3-sonnet-20240229-v1:0"
response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": "What should I know about elves?"
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": model_id,
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

## 3. Amazon Bedrock용 Guardrail 구성

Knowledge Base를 생성하고 문서를 동기화했으니, 이제 Amazon Bedrock Guardrail을 생성하겠습니다. 

Contextual grounding 검사를 위한 필터링 파라미터는 두 가지입니다:

- **Grounding** – 모델 응답이 근거(source)에 기반한 사실인지 확인하기 위한 최소 신뢰도 점수(임계값)를 설정해 활성화할 수 있습니다. 근거 점수와 동일하거나 높은 응답만 허용되며, 임계값보다 낮은 경우 응답이 차단되고 구성한 차단 메시지가 반환됩니다.

- **Relevance** – 사용자 질의와의 관련성을 위한 최소 신뢰도 점수를 기준으로 동작합니다. 설정한 관련성 임계값보다 낮은 응답은 차단되며, 해당 차단 메시지가 반환됩니다.

Grounding과 relevance 임계값을 높게 설정할수록 더 많은 응답이 차단됩니다. 사용 사례에서 허용 가능한 정확도 수준에 따라 값을 조정하세요. 예를 들어 금융 도메인의 고객용 애플리케이션은 부정확한 콘텐츠에 대한 허용 범위가 낮으므로 높은 임계값이 필요할 수 있습니다.

In [ ]:
response = bedrock.create_guardrail(
    name="contextual-grounding-guardrail-{}".format(unique_id),
    description="D&D Guardrail",
    contextualGroundingPolicyConfig={
        'filtersConfig': [
            {
                'type': 'GROUNDING',
                'threshold': 0.5
            },
            {
                'type': 'RELEVANCE',
                'threshold': 0.8
            },
        ]
    },
    blockedInputMessaging="Sorry, I can not respond to this.",
    blockedOutputsMessaging="Sorry, I can not respond to this.",
)
guardrailId = response["guardrailId"]
print("The guardrail id is",response["guardrailId"])

## 4. Contextual grounding 기능 테스트
Knowledge Base와 Guardrail 구성을 마쳤으니 함께 테스트해 보겠습니다.

이 섹션에서는 먼저 Knowledge Base에서 결과를 검색한 뒤 Guardrail이 통합된 Converse API에 전달합니다.

In [13]:
def invoke_kb(kb_query):
    kb_response = bedrock_agent_runtime_client.retrieve(
        knowledgeBaseId=kb_id,
        retrievalConfiguration={
            'vectorSearchConfiguration': {
                'numberOfResults': 2,
            }
        },
        retrievalQuery={
            'text': kb_query
        }
    )
    model_id="anthropic.claude-3-sonnet-20240229-v1:0"
 
    inference_config = {"temperature": 0.1}

    # The message for the model and the content that you want the guardrail to assess.
    messages = [
        {
            "role": "user",
            "content": [
                {"text": str(kb_response)},
                {"text": kb_query}
            ]
        }
    ]
    response = bedrock_runtime.converse(modelId=model_id,messages=messages, inferenceConfig=inference_config)
    print("""
    ================================
    Invoke KB without Guardrails
    ================================
    """)
    print_results(kb_response, response)


def invoke_kb_with_guardrail(kb_query):
    kb_response = bedrock_agent_runtime_client.retrieve(
        knowledgeBaseId=kb_id,
        retrievalConfiguration={
            'vectorSearchConfiguration': {
                'numberOfResults': 2,
            }
        },
        retrievalQuery={
            'text': kb_query
        }
    )
    model_id="anthropic.claude-3-sonnet-20240229-v1:0"
    inference_config = {"temperature": 0.1}
    guardrail_config = {
        "guardrailIdentifier": guardrailId,
        "guardrailVersion": "DRAFT",
        "trace": "enabled"
    }

    # The message for the model and the content that you want the guardrail to assess.
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "guardContent": {
                        "text": {
                            "text": str(kb_response),
                            "qualifiers": ["grounding_source"],
                        }
                    }
                },
                {
                    "guardContent": {
                        "text": {
                            "text": kb_query,
                            "qualifiers": ["query"],
                        }
                    }
                },
            ],
        }
    ]
    response = bedrock_runtime.converse(modelId=model_id,messages=messages,guardrailConfig=guardrail_config, inferenceConfig=inference_config,)
    print("""
    ================================
    Invoke KB with Guardrails
    ================================
    """)
    print_results_with_guardrail(kb_response, response)


In [ ]:
kb_query = "What are High Elves?"
invoke_kb(kb_query)
invoke_kb_with_guardrail(kb_query)

In [ ]:
kb_query = "Where should the elves go if they arrive in Paris?"
invoke_kb(kb_query)
invoke_kb_with_guardrail(kb_query)

## 5. 리소스 삭제
불필요한 비용을 피하기 위해 모든 리소스를 삭제합니다. 

In [ ]:
## Delete the Knowledge Base
knowledge_base.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)
## Delete the Guardrail
bedrock.delete_guardrail(guardrailIdentifier = guardrailId)